![Podmanlogo](Pictures/podman-logo.png)

# From Podman to Kubernetes and viceversa

When working with kubernetes what most users want is a way of easily deploying what they have already tested in Podman. This can be done easily as Podman helps generating a yaml file that can be used afterwards in kubernetes. We demonstrated this on the Podman 101 workshop, but we are going to do it again with our Patient Portal application. Then we'll deploy it to an actual kubernetes cluster (in our case we chose OpenShift distribution).

Log into the system:

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Podman201'] }}

Lets begin by creating all the containers of our application in Podman. As always, you need to start by creating the networks and the volume:

In [ ]:
podman network create database
podman network create payment
podman volume create patient-portal-data

Now we can spin up all of our three containers. We added a sleep in the command list to give enough time for the database to initialize before spining up the frontend container, otherwise the deployment of the app would fail.

In [ ]:
podman run -d --rm --name database --network database -v patient-portal-data:/pgdata quay.io/skupper/patient-portal-database
sleep 10
podman run -d --rm --name payment-processor --network payment quay.io/skupper/patient-portal-payment-processor
podman run -d --rm --name frontend --network payment,database -p 8080:8080 \
-e DATABASE_SERVICE_HOST="database" \
-e DATABASE_SERVICE_PORT="5432" \
-e PAYMENT_PROCESSOR_SERVICE_HOST="payment-processor" \
-e PAYMENT_PROCESSOR_SERVICE_PORT="8080" \
quay.io/skupper/patient-portal-frontend

Check everything is working as expected:

In [ ]:
podman ps -a

Check the logs:

In [ ]:
podman logs frontend

Run an http request:

In [ ]:
curl -s localhost:8080

Everything looks fine, it's time to create a yaml file to be used in kubernetes out of our running containers. For that we can simply use the "podman generate kube" command. Run it for the database container and review the output:

In [ ]:
podman generate kube --replicas 1 --type deployment database > database.yml
cat database.yml

We used the "--type deployment" option to specify we want to use a Deployment, otherwise it would have just defined a pod. Moreover you can set the number of replicas of your pod with the "--replicas" option.

> **Note**: you can also use the "--service" option and the command will generate the definition for the service in your yaml file as well as the deployment. We're not using it because it generates a service of the type NodePort which is not the default in kubernetes, we want to use ClusterIP services in our kubernetes cluster so we will create the services manually afterwards.
>
> For now the only thing you need to know is that NodePort services are used for accessing to the application from outside of the cluster through to the kubernetes node. On the other hand ClusterIP is used for intra cluster communication. We will manually create the services for payment-processor and database as we want them to be internal to the cluster while we will use the NodePort service generated by Podman for the frontend as we want it to be exposed outside of the cluster. If you want to learn more about the different types of services take a look at [the official kubernetes documentation](https://kubernetes.io/docs/concepts/services-networking/service/).

As we are pointing the command "podman generate kube" command to a single container it will consider that we want to create a pod with a single container in our kubernetes cluster. We could also point to a Podman pod and it would generate a deployment with multiple containers in a pod if that is what we had in the Podman pod.

Continue with the payment-processor container:

In [ ]:
podman generate kube --replicas 3 --type deployment payment-processor > payment-processor.yml
cat payment-processor.yml

See how we specified we want three replicas of this service and, in the yaml file, it's also defined under the "spec.replicas" part of the Deployment definition.

And, last, for the frontend:

In [ ]:
podman generate kube --replicas 1 --service --type deployment frontend > frontend.yml
cat frontend.yml

For the frontend we have used the "--service" option because we can use a Service of type NodePort as it will be exposed outside of the cluster. Now, lets deploy all these workloads to our kubernetes cluster. Log into the OpenShift cluster:

In [ ]:
oc login -u student{{ STDID}} -p {{ PASSSTU }} --insecure-skip-tls-verify https://{{ OCENDPOINT }}:6443

Create a new project for our application:

In [ ]:
oc new-project patient-portal-student{{ STDID}}

Begin by deploying the payment processor container:

In [ ]:
oc apply -f payment-processor.yml -n patient-portal-student{{ STDID}}

Check it's been succesfully deployed:

In [ ]:
oc get deployment,pods

You see all of our resources have been deployed in a super easy way.

We're going to deploy our database now as it has to be initialized before the frontend for our application to work.

In [ ]:
oc apply -f database.yml -n patient-portal-student{{ STDID}}

Check the status of our newly created database workload, use the "-l" to filter for the label "app" with a value of "database-pod":

In [ ]:
oc get pods,deployment -l app=database-pod

The pod status is "Pending", lets find why. Take a look at the events in our OpenShift project by using the "oc get events" conmmand. Filter it to the last events only:

In [ ]:
oc get events | head -n 5

You can see in the last event (the one on top) that our pod failed scheduling because it cannot find a persistent volume for our database in any node of the cluster. This is normal because we don't have any storage configured in our OpenShift cluster.

To solve this issue you'd need to create a Persistent Volume in kubernetes, as we don't have a storage provider we will just create our pod with no persistent storage. Let's modify our database.yml file, the following command deletes all the lines at the end of the file which are the ones describing the volume and volume mount:

In [ ]:
sed -i '33,$ d' database.yml

Now we want to force the replacement of the resources we created before with the new definitions, delete the previous deployment and apply the file again:

In [ ]:
oc delete deployment database-pod-deployment
oc apply -f database.yml

Again check the status of our database workload:

In [ ]:
oc get pods,deployment -l app=database-pod

Sometimes you'll see the status of the container as "Running", but if you run the previous command again you'll at some point see its status as "CrashLoopBackOff". This means the configuration is correct from the kubernetes perspective but there is something wrong in the container or container image itself.

Take a look at the last events in case we can find something:

In [ ]:
oc get events | head -n 5

Everything looks fine, it pulled the container image and scheduled it to be deployed. Use the "oc logs" command to find what's going on inside the container:

In [ ]:
export PODNAME=$(oc get pod -l app=database-pod -o jsonpath="{.items[0].metadata.name}")
oc logs $PODNAME

Thanks to the logs we found there is a permissions problem in the container. The postgresql process is trying to initialize the database in a directory that is reserved for priviledged users only, we can solve it by creating the database in a different directory. We looked at the documentation of this container image and it seems that this can be easily done by using a environment variable during the deployment of the container.

I want to remark here that OpenShift is a platform build with a huge focus on security and this is a good example of it. The user inside our container is not priviledged so it cannot access to priviledged directories. OpenShift counts with different ways of controling security, one of the most widely used is SCC. We'll not cover this in this workshop, but you can take a look at  [the official documentation](https://access.redhat.com/documentation/en-us/openshift_container_platform/4.14/html/security_and_compliance/container-security-1#security-deployment-sccs_security-platform).

Going back to our database container, we now know we need to set the environment variable "PGDATA" with a value that points to a user created directory, we'll use "pgdata/patient-portal". We can achieve this by adding the environment variable to the database.yml file, but in our case we're going to show how to use the "oc set" command:

In [ ]:
oc set env deployment/database-pod-deployment PGDATA=/pgdata/patient-portal

That command will modify the yaml file stored by kubernetes and add the environment variable and its value to it. This will automatically trigger a recreation of the container as kubernetes consider this addition important enough to recreate it. You can confirm this by looking at the events:

In [ ]:
oc get events | head -n 5

In the left column you see how long ago the events were triggered, you should see the pod was scheduled when you ran the "oc set" command. In case you want to force a redeployment of your application, just to be 100% sure it's redeployed, you can use the "oc rollout restart" command:

In [ ]:
oc rollout restart deployment/database-pod-deployment

Now everything should be working as expected, lets review the logs again:

In [ ]:
export PODNAME=$(oc get pod -l app=database-pod -o jsonpath="{.items[0].metadata.name}")
oc logs $PODNAME

Everything is working fine, we have troubleshooted the deployment of our database!

> **Note**: as you can see it took some work to deploy the database container, we could have just configured the Podman container so it doesn't use a volume and uses the environment variable in Podman as well. With that we could generate a yaml file to use with kubernetes without any troubleshooting. We decided to not doing that so we can just show how to troubleshoot this small issues. In general, if you know your application is going to be deployed in kubernetes you may apply some changes to it when working in Podman so it can afterwards be easily deployed in your kubernetes cluster.

Last, but not least, we need to deploy our frontend container:

In [ ]:
oc apply -f frontend.yml -n patient-portal-student{{ STDID}}

Check everything has been correctly deployed:

In [ ]:
oc get pods,deployment,service -l app=database-pod

Everything looks fine, lets review the logs of our application:

In [ ]:
export PODNAME=$(oc get pod -l app=frontend-pod -o jsonpath="{.items[0].metadata.name}")
oc logs $PODNAME

We observe the frontend cannot reach the other microservices. This happens because the database and payment processor microservices are not exposed with a Service object. Lets expose them:

In [ ]:
oc expose deployment/database-pod-deployment --port 5432 --name database
oc expose deployment/payment-processor-pod-deployment --port 8080 --name payment-processor

> **Note**: our application is hardcoded to look for the "database" and "payment-processor" DNS names. This is why we have to use the "--name" option for the services. Otherwise our frontend pods wouldn't be able to reach the others. Remember the Service name is the DNS name used to reach your microservices.

Check both are present:

In [ ]:
oc get services

Now restart the frontend as it need to get in touch with the other microservices at boot:

In [ ]:
oc rollout restart deployment/frontend-pod-deployment

Review the logs again:

In [ ]:
export PODNAME=$(oc get pod -l app=frontend-pod -o jsonpath="{.items[0].metadata.name}")
oc logs $PODNAME

Our application is now working, but not accessible from outside of the cluster. Expose the frontend service to access to it from outside of kubernetes:

In [ ]:
oc expose service/frontend-pod

Check your route has been created:

In [ ]:
oc get route

Check it's accessible running a curl command:

In [ ]:
export ROUTEADDRESS=$(oc get route frontend-pod -o jsonpath={.spec.host})
curl -s $ROUTEADDRESS

And it works just as expected!

> **Note**: in this excercise we have used different yaml files for each microservice. Remember you can use a single file for all of them, you just need to separate each definition with the "---" line.

We've seen how you can use Podman to generate the files that you'll use later to deploy your workloads in kubernetes. And probably you're thinking that kubernetes yaml files are super cool and you'd like to use them also for Podman workloads. No worries, Podman got your back.

Before moving on, stop all of the running containers:

In [ ]:
podman rm --all -f

Now, if you wanted to run your pod in Podman using the kubernetes yaml file you just need to run the following command pointing to it:

In [ ]:
podman kube play database.yml

If you check the output, it generates a pod with the container within it. This is because in kubernetes we always have a pod and Podman just follows what's in the yaml file.

In [ ]:
podman ps -a --pod

Deploy the payment processor and the frontend using their kubernetes yaml files.

In [ ]:
podman kube play payment-processor.yml
podman kube play frontend.yml

First thing you notice is a warning. Podman's telling us that the amount of replicas for the payment-processor workload has been reduced to 1. This is because Podman is not designed for scalability, it's a single node solution. Therefore it doesn't make sense to have multiple replicas.

Review all containers are up and running:

In [ ]:
podman ps -a --pod

We have now 6 containers, this is because we have 3 pods and Podman deploys a pod infrastructure management container in every pod.

Review frontend logs (we need to use the automatically generated name for the container):

In [ ]:
podman logs frontend-pod-deployment-pod-frontend

Also check if the app receives http requests:

In [ ]:
curl -s localhost:8080

Everything is working perfectly and using yaml files!

There is one last integration between Podman and kubernetes yaml files that I want to mention. Remember Quadlets? They are pretty similar to kubernetes yaml files, your container is defined in them and systemd deploys the containers. But isn't it duplication of work to have both, a Quadlet and a kubernetes yaml file, for the same container? It is! And that is why you can have Quadlets directly pointing to kubernetes yaml files instead of having to define the whole container.

Before creating the Quadlets, stop all the Podman workloads.

In [ ]:
podman pod rm --all -f
podman rm --all -f

Check nothing is running:

In [ ]:
podman ps -a

Lets see how these Quadlets would look like. Start with the database one:

In [ ]:
cat << EOF > database.kube
[Install]
WantedBy=default.target

[Kube]
# Point to the yaml file in the same directory
Yaml=database.yml
EOF
cat database.kube

As you can see we just define the target, as always, and then we point to our yaml file in the "[Kube]" section. As easy as it is!
Rembember we are using a relative path to the yaml file.

Do the same for the payment processor container:

In [ ]:
cat << EOF > payment-processor.kube
[Install]
WantedBy=default.target

[Unit]
Requires=database.service
After=database.service

[Kube]
# Point to the yaml file in the same directory
Yaml=payment-processor.yml
EOF

We added the "[Unit]" section to guarantee everything is booted in the right order.

And for the frontend container:

In [ ]:
cat << EOF > frontend.kube
[Install]
WantedBy=default.target

[Unit]
Requires=payment-processor.service
After=payment-processor.service

[Kube]
# Point to the yaml file in the same directory
Yaml=frontend.yml
EOF

Move all the files to "~/.config/containers/systemd/". You need to move your yaml file as we used a relative path in the Quadlet definition.

In [ ]:
mv ~/* ~/.config/containers/systemd/

Now that all the files are present in the correct directory you just need to reload the systemctl daemon:

In [ ]:
systemctl --user daemon-reload

Finally, use systemctl to manage your containerized workload as a systemd unit:

In [ ]:
systemctl --user start database.service
systemctl --user start payment-processor.service
systemctl --user start frontend.service

Review all of your containers and pods are up and running:

In [ ]:
podman ps -a --pod

Check frontend logs:

In [ ]:
podman logs frontend-pod-deployment-pod-frontend

And make sure your app is answering http requests:

In [ ]:
curl -s localhost:8080

As you can see everything works perfectly!

We have seen many ways in which Podman and Kubernetes are interconnected. Hopefully you can leverage all these interconnections to have a better integration and usability in your cloud-native environments.

# Podman Desktop and kubernetes

Podman Desktop provides a graphical interface to work with Podman, but the most interesting part is the extensions it uses to easily integrate with your kubernetes clusters.

Review [this blog post](https://developers.redhat.com/articles/2023/11/06/working-kubernetes-podman-desktop#working_with_remote_kubernetes_clusters) to know more about it!

# Clean up

In [ ]:
podman rm --all -f
podman network prune -f
podman volume prune -f
podman pod prune -f
podman image prune -f
rm -rf ~/.config/containers/systemd/*
rm database.yml frontend.yml payment-processor.yml
oc delete project patient-portal-student{{ STDID}}

In [ ]:
%logout

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#631f61;"></i>&nbsp;&nbsp;Next Steps

# Conclusion

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="7-WKSHP-Conclusion.ipynb" target="New" title="Conclusion"><i class="fas fa-chevron-circle-right" style="color:#631f61;"></i></a></h2>

</br>
 <a href="5-WKSHP-Introduction-to-kubernetes.ipynb" target="New" title="Back: Introduction to Kubernetes"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="7-WKSHP-Conclusion.ipynb" target="New" title="Conclusion"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
